In [1]:
import os
import sys

sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"
import pandas as pd
import numpy as np

from quantfreedom.class_practice.enums import *
from quantfreedom.class_practice.base import backtest_df_only


np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

price_data = pd.read_hdf("../../tests/data/4hBTCETH.hd5")
%load_ext autoreload
%autoreload 2


In [2]:
entries_array = np.array(
    [
        [False, False, True, True, False, True],
        [True, True, True, True, False, False],
    ]
).T
entries = pd.DataFrame(entries_array, index=price_data.index)

In [3]:
account_state = AccountState()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01]) / 100,
    sl_based_on_lookback=np.array([30]),
    risk_reward=np.array([2, 5]) * 1.0,
    leverage_type=np.array([LeverageType.Dynamic]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    stop_loss_type=np.array([StopLossType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitType.RiskReward]),
    max_equity_risk_pct=np.array([100]) / 100,
    order_type=np.array([OrderType.Long]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.High]),
    sl_to_be_when_pct_from_candle_body=np.array([1, 2]) / 100,
    sl_to_be_zero_or_entry=np.array([SLToBeZeroOrEntryType.ZeroLoss]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.Close]),
    trail_sl_when_pct_from_candle_body=np.array([2.0]) / 100,
    trail_sl_by_pct=np.array([1.0]) / 100,
)
backtest_settings = BacktestSettings()
exchange_settings = ExchangeSettings()

In [9]:
backtest_df_only(
    account_state=account_state,
    order_settings_arrays=order_settings_arrays,
    backtest_settings=backtest_settings,
    exchange_settings=exchange_settings,
    price_data=price_data,
    entries=entries,
)


Creating cartesian product ... after this the backtest will start, I promise :).

Starting the backtest now ... and also here are some stats for your backtest.

Total symbols: 2
Total indicator settings per symbol: 1
Total indicator settings to test: 2
Total order settings per symbol: 4
Total order settings to test: 8
Total candles per symbol: 6
Total candles to test: 48

Total combinations to test: 8

New Symbol

New Indicator Setting

New Order Setting

Checking Next Bar for entry or exit

Checking Next Bar for entry or exit
Long Order - Calculate Stop Loss - calculate_stop_loss
Long Order - Candle Body Getter - __get_candle_body_price_low
Long Order - Increase Position - __get_possible_loss
Long Order - Increase Position - risk_pct_of_account_and_sl_based_on_not_in_pos
Long Order - Increase Position - __check_size_value
Long Order - Calculate Leverage - calculate_dynamic_leverage
Long Order - Calculate Leverage - __calc_liq_price
Long Order - Calculate Take Profit - calculate_risk_r